# SQLite DB Access
Basic access to a SQLite database, through JDBC.

### Using Maven to get to the JDBC driver

In [1]:
// Proxy settings
System.out.println(String.format("HTTP Proxy: %s", System.getProperty("HTTP_PROXY")));
System.out.println(String.format("HTTPS Proxy: %s", System.getProperty("HTTPS_PROXY")));

HTTP Proxy: null
HTTPS Proxy: null


In [2]:
%maven org.xerial:sqlite-jdbc:3.34.0

In [3]:
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.Driver;
import java.sql.DatabaseMetaData;
import java.sql.PreparedStatement;
import java.sql.ResultSet;
import java.sql.ResultSetMetaData;
import java.sql.Statement;
import java.sql.SQLException;
import java.util.stream.Collectors;

In [4]:
Connection connection;
try {
    Class.forName("org.sqlite.JDBC");
    String dbURL = "jdbc:sqlite:../../../../oliv.db"; // oliv.db: db file location on the machine jupyter runs on.
    connection = DriverManager.getConnection(dbURL);
    if (connection != null) {
        System.out.println("Connected to the database");
        DatabaseMetaData md = connection.getMetaData();
        System.out.println("Driver name: " + md.getDriverName());
        System.out.println("Driver version: " + md.getDriverVersion());
        System.out.println("Product name: " + md.getDatabaseProductName());
        System.out.println("Product version: " + md.getDatabaseProductVersion());
    }
} catch (ClassNotFoundException ex) {
    ex.printStackTrace();
} catch (SQLException ex) {
    ex.printStackTrace();
}


Connected to the database
Driver name: SQLite JDBC
Driver version: 3.34.0
Product name: SQLite
Product version: 3.34.0


In [5]:
String sqlStmt = "SELECT * FROM OLIV_TABLE";

In [6]:
ResultSet rs;
PreparedStatement preparedStatement = null;
Statement statement = null;

In [7]:
statement = connection.createStatement();
rs = statement.executeQuery(sqlStmt);

In [8]:
ResultSetMetaData metaData = rs.getMetaData();
int columnCount = metaData.getColumnCount();
System.out.println(String.format("Returned %d columns", columnCount));

Returned 3 columns


In [9]:
int nbRows = 0;
while (rs.next()) {
    nbRows++;
    for (int i = 0; i < columnCount; i++) {
        String colName = metaData.getColumnName(i + 1);
        Object colValue = rs.getObject(i+1);
        System.out.println(String.format("Row #%d, %s: %s", nbRows, colName, colValue));
    }
}

Row #1, id: 1
Row #1, data: Hi there
Row #1, date: 2021-03-29 14:37:16


_Done with the Query_  
Closing resources.  
> Note: This could very well be done - in Java - with a `try with resources`.   
> This is done explicitely here, for clarity.

In [10]:
rs.close();
statement.close();

**Using a `try with Resources`, as an example**

In [11]:
try (Statement statement = connection.createStatement()) {
    try (ResultSet resultSet = statement.executeQuery("SELECT * FROM OLIV_TABLE")) {

        ResultSetMetaData metaData = resultSet.getMetaData();
        int columnCount = metaData.getColumnCount();

        while (resultSet.next()) {

            List<Object> oneRow = new ArrayList<>();
            for (int i = 0; i < columnCount; i++) {
                oneRow.add(resultSet.getObject(i + 1));
            }
            System.out.println(oneRow.stream().map(obj -> String.valueOf(obj)).collect(Collectors.joining(", ")));
        }
    }
}

1, Hi there, 2021-03-29 14:37:16


In [12]:
connection.close();
System.out.println("Bye");

Bye


---